## Importações

In [2]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader
from pandas.util.testing import assert_frame_equal #import alterado

from tqdm import tqdm_notebook, tqdm
from collections import deque

/tmp/ipykernel_24371/2316383207.py:8: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal #import alterado


In [3]:
tf.__version__

'2.4.1'

In [4]:
class IA_Trader():
    """
    state_size = estados que estão vindo do ambiente, por exemplo os sensores de
     um carro autônomo. Seria a camada de entrada da rede neural.

    action_space = indica o numero de ações que o algoritmo pode tomar, ou seja,
    suas escolhas. Comprar, vender, ou não fazer nada.

    model_name = Nome do modelo de aprendizagem.
    """
    def __init__(self, state_size, action_space = 3, model_name="IATrader"):
        self.state_size = state_size
        self.action_space = action_space
        # memmoria de 2000 ações, ou seja, o modelo não precisa ficar treinando algumas etapas repetitivas
        self.memory = deque(maxlen = 2000)
        #memoria da experiencia de replay
        self.model_name = model_name

        #gama da equação de bellmam = fator de desconto
        self.gamma = 0.95

        #indica se as ações serão tomadas randomicamente ou pela rede neural. Sendo 1.0 = 100% randomico
        self.epsilon = 1.0

        #porcentegem do modelo tomar uma ação randomica, ou seja, com o valor de 0.1 o 
        #agente tem 10% de chance de tomar uma ação randomica e fazer ele explorar mais o ambiente.
        self.epsilon_final = 0.01

        #variavel responsável por decrementar o valor de epsilon, para poder fazer o ajuste do peso da rede.
        self.epsilon_decay = 0.995

        #criando o modelo quando o objeto é chamado
        self.model = self.model_builder();

    #construção da rede neural
    def model_builder(self):
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
        model.add(tf.keras.layers.Dense(units=64, activation='relu'))
        model.add(tf.keras.layers.Dense(units=128, activation='relu'))
        model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear')) #linear retorna todos os voleres dos neurônios
        model.compile(loss='mse', optimizer = tf.keras.optimizers.Adam(learning_rate=0.001))
        return model

    #função que irá fazer toda a negociação, ou seja, as ações que serão tomadas
    def trade(self, state):
        #função que irá tomar uma decisão aleatória no ambiente caso o valor passado pela função random for menor que a de epsilon, possibilitando a exploração do ambiente
        if random.random() <= self.epsilon:
            return random.randrange(self.action_space)

        #caso a ação tomada não for aleatória, ou seja, não acionando a condicional anterior, os valores serão buscados por meio da rede neural
        actions = self.model.predict(state)
        #retorna o maior valor dita pela neural
        return np.argmax(actions[0])
    
    #seleciona um grupo de dados para fazer o treinamento
    def batch_train(self, batch_size):
        batch = []

        #pegando somente as amostra do final da memória. Pois fica sem sentido, no caso das ações, pegar uma ação aleatória de 2005 e usar para prever outra em 2020
        for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
            batch.append(self.memory[i])

        for state, action, reward, next_state, done in batch:
            if not done:
                reward = reward + self.gamma * np.amax(self.model.predict(np.asarray(next_state))[0]) #pega os valores de Q para o prox estado

            #pegando os valores de Q para o estado atual
            target = self.model.predict(state)
            #Adicionando uma recompensa para a ação, onde a coluna [action] recebe o valor dessa recompensa.
            target[0][action] = reward

            self.model.fit(state, target, epochs=1, verbose = 0)

        #Atualizando o epislon, caso ele for maior que o [epsilon_final]
        if self.epsilon > self.epsilon_final:
            self.epsilon *= self.epsilon_decay


Equação de belmam utilizada para calcular a recompensa

# $$V(s) = max_{a}(R(s, a)) + \gamma V(s')) | $$

# $$\gamma = 0.9 $$


Parte utilizada na recompensa consiste:
# $$ R(s, a)) + \gamma V(s') $$

onde $R(s, a)$ = reward = recompensa no estado atual;

$ \gamma V(s') $ = self.gamma * np.amax(self.model.predict(next_state)[0])

$ s' $ = proximo estado.


#### Sigmoid

## $ y = \frac{1}{1 + e^{-x}} $

In [5]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [6]:
def stocks_price_format(n):
    if n < 0:
        #caso for um valor negativo, adiciona o menos antes do preço($)
        return "- $ {0:2f}".format(abs(n))
    else:
        #se não fica sem o menos
        return "$ {0:2f}".format(abs(n))

In [7]:
dataset = data_reader.DataReader("AAPL", data_source = "yahoo")

In [8]:
dataset.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-12-29,29.277500,29.100000,29.112499,29.182501,60158000.0,27.468958
2016-12-30,29.299999,28.857500,29.162500,28.955000,122345200.0,27.254812
2017-01-03,29.082500,28.690001,28.950001,29.037500,115127600.0,27.332472
2017-01-04,29.127501,28.937500,28.962500,29.004999,84472400.0,27.301880
2017-01-05,29.215000,28.952499,28.980000,29.152500,88774400.0,27.440722


In [9]:
str(dataset.index[0]).split()[0]

'2016-12-29'

In [10]:
def dataset_loader(stock_name):
    #lendo o dataset, onde busca-se pela ação [stock_name] na base de dados do yahoo
    dataset = data_reader.DataReader(stock_name, data_source = "yahoo")
    #data de inicio da ação. Pode ser também em um periodo específico
    start_date = str(dataset.index[0]).split()[0]
    #ultima data da ação coletada
    end_date = str(dataset.index[-1]).split()[0]
    #preço de fechamento da ação no dia
    close = dataset['Close']
    return close

In [11]:
def state_creator(data, timestep, window_size):
    """
    Função onde pega os dados da base de dados de acordo com o timestep (episodios do treinamento)
    e widow_size (tamanho do janelamento dos dados)
    
    """
    starting_id = timestep - window_size + 1
  
    if starting_id >= 0:
        # se o indice da ação for maior do que 0, então ele pega os valores de acordo com o window_size
        windowed_data = data[starting_id:timestep + 1]
    else:
        # se caso o index for 0, ele replica o mesmo valor de acordo com o tamanho do window_size
        windowed_data = - starting_id * [data[0]] + list(data[0:timestep + 1])
    
    state = []
    for i in range(window_size - 1):
        # normalização dos valores atraves da função da sigmoid = valores entre 0 e 1
        state.append(sigmoid(windowed_data[i + 1] - windowed_data[i]))
    
    return np.array([state])

### Escolhendo a ação a ser feita o trade pelo bot

In [12]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

In [13]:
state_creator(data, 20, 5)

array([[0.61715713, 0.50375025, 0.50062466, 0.48001068]])

In [20]:
window_size = 10
episodes = 2
batch_size = 32
data_samples = len(data) - 1

In [21]:
data_samples

1257

### Chamando o modelo construido

In [22]:
trader = IA_Trader(window_size)

In [23]:
trader.model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                352       
_________________________________________________________________
dense_5 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_6 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Realizando o treinamento

In [24]:
for episode in range(1, episodes + 1):
    print("Episode: {}/{}".format(episode, episodes))
    #criando os estados com 10 valores, sendo o [window_size =- 1], então para 10 valores adiciona +1
    # começando com a primeira posição do dataset = 0   
    state = state_creator(data, 0, window_size + 1)
    #total de lucros com a compra
    total_profit = 0
    #armazena todas as ações que será comprada
    trader.inventory = []

    # percorrendo a base de dados 
    for t in tqdm(range(data_samples)):
        # ação que será tomada
        action = trader.trade(state)
        # cálculo da próxima ação
        next_state = state_creator(data, t + 1, window_size + 1)
        # inicialização das recompensas
        reward = 0

        # Comprando uma ação se for 1
        if action == 1: 
            # guardando os valores  
            trader.inventory.append(data[t])
            print("AI Trader bought: ", stocks_price_format(data[t]))

        # Vendendo acao se for 2 e verifica se já foi comprado alguma ação
        elif action == 2 and len(trader.inventory) > 0:
            # retira a ação e passa para a variavel  
            buy_price = trader.inventory.pop(0)
            
            # calculo da recompensa, onde é definida pelo valor da venda menos o valor comprado, se for negativo, retorna 0 = sem recompensa
            reward = max(data[t] - buy_price, 0)
            # cálculo do lucro  
            total_profit += data[t] - buy_price
            print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price))
            
        # significa que está no ultimo dado da base de dado
        if t == data_samples - 1:
            done = True
        else:
            done = False

        # armazenando na memoria os dados
        trader.memory.append((state, action, reward, next_state, done))

        # passando para o proximo estado
        state = next_state

        if done:
            print("########################")
            print("Total profit: {}".format(total_profit))
            print("########################")

        # quando o tamanho da memoria for maior que o batch size, faz-se o treinamento do modelo
        if len(trader.memory) > batch_size:
            trader.batch_train(batch_size)
     
    # salvando o modelo a cada 10 episodios
    if episode % 10 == 0:
        trader.model.save("ai_trader_{}.h5".format(episode))
    

Episode: 1/2


AI Trader bought:  $ 29.182501
AI Trader sold:  $ 29.037500  Profit: - $ 0.145000
AI Trader bought:  $ 29.777500
AI Trader bought:  $ 29.937500
AI Trader sold:  $ 29.812500  Profit: $ 0.035000
AI Trader sold:  $ 30.000000  Profit: $ 0.062500
AI Trader bought:  $ 29.992500
AI Trader bought:  $ 30.485001
AI Trader sold:  $ 32.187500  Profit: $ 2.195000
AI Trader sold:  $ 32.132500  Profit: $ 1.647499
AI Trader bought:  $ 33.009998
AI Trader bought:  $ 33.029999
AI Trader sold:  $ 33.322498  Profit: $ 0.312500
AI Trader bought:  $ 33.755001


Exception ignored in: <function tqdm.__del__ at 0x7f345e51e4c0>
Traceback (most recent call last):
  File "/home/lps/anaconda3/envs/ia/lib/python3.8/site-packages/tqdm/std.py", line 1152, in __del__
    self.close()
  File "/home/lps/anaconda3/envs/ia/lib/python3.8/site-packages/tqdm/notebook.py", line 286, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


AI Trader sold:  $ 33.930000  Profit: $ 0.900002


AI Trader sold:  $ 34.132500  Profit: $ 0.377499


...

AI Trader bought:  $ 43.825001


 24%|██▍       | 306/1257 [08:12<26:56,  1.70s/it]

AI Trader bought:  $ 43.810001


 24%|██▍       | 307/1257 [08:14<26:10,  1.65s/it]

AI Trader bought:  $ 42.817501
